# Packages

In [56]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [230]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from auto_ml import Predictor

# Data

In [217]:
dateparser = lambda x: pd.datetime.strptime(x, '%d/%m/%Y %H:%M')

X_train = pd.read_csv('./X_train_v2.csv', header=0, index_col=0, parse_dates=['Time'], date_parser=dateparser)
y_train = pd.read_csv('./Y_train_sl9m6Jh.csv', header=0, index_col=0)

X_test = pd.read_csv('./X_test_v2.csv', header=0, index_col=0, parse_dates=['Time'], date_parser=dateparser)

X_train.head(3)

WF                Time  NWP1_00h_D-2_U  NWP1_00h_D-2_V  NWP1_00h_D-2_T  \
ID                                                                            
1   WF1 2018-05-01 01:00:00             NaN             NaN             NaN   
2   WF1 2018-05-01 02:00:00             NaN             NaN             NaN   
3   WF1 2018-05-01 03:00:00             NaN             NaN             NaN   

    NWP1_06h_D-2_U  NWP1_06h_D-2_V  NWP1_06h_D-2_T  NWP1_12h_D-2_U  \
ID                                                                   
1              NaN             NaN             NaN             NaN   
2              NaN             NaN             NaN             NaN   
3              NaN             NaN             NaN             NaN   

    NWP1_12h_D-2_V  ...  NWP4_00h_D-1_CLCT  NWP4_12h_D-1_U  NWP4_12h_D-1_V  \
ID                  ...                                                      
1              NaN  ...                NaN             NaN             NaN   
2              NaN  ...                NaN             NaN             NaN   
3              NaN  ...                NaN             NaN             NaN   

    NWP4_12h_D-1_CLCT  NWP4_00h_D_U  NWP4_00h_D_V  NWP4_00h_D_CLCT  \
ID                                                                   
1                 NaN      1.254603     -0.289687        82.543144   
2                 NaN      2.490908     -0.413370        99.990844   
3                 NaN      0.997093     -1.415138        98.367235   

    NWP4_12h_D_U  NWP4_12h_D_V  NWP4_12h_D_CLCT  
ID                                               
1            NaN           NaN              NaN  
2            NaN           NaN              NaN  
3            NaN           NaN              NaN  

[3 rows x 104 columns]

# Base line

## WF1

In [221]:
target = 'Production'
features = [i for i in X_train.columns if ((i != 'WF')& (i != 'Time') )]

wf_name = 'WF1'

def create_data_WFx(wf_name, X_train, y_train=None):
    df_train = pd.concat([y_train, X_train], axis=1)
    df_train_WFx = df_train[df_train.WF == wf_name]
    df_train_WFx.index = df_train_WFx.loc[:, 'Time']
    df_train_WFx.drop(['Time'], axis=1,inplace=True)
    
    return df_train_WFx

df_train_WFx = create_data_WFx(wf_name, X_train, y_train)
df_test_WFx = create_data_WFx(wf_name, X_test)


In [224]:
def train_wf(df_train_WFx):
    column_descriptions = {
        target: 'output',
        'WF': 'ignore'
    }
    model_names=[
        'GradientBoostingRegressor',
        'LGBMRegressor',
        'RandomForestRegressor', 
        'LinearRegression', 
        'AdaBoostRegressor', 
        'ExtraTreesRegressor', 
        'RANSACRegressor', 
        'Lasso', 
        'MiniBatchKMeans',
        'Ridge',
        'PassiveAggressiveRegressor'
    ]
    ml_predictor = Predictor(
        type_of_estimator='regressor', 
        column_descriptions=column_descriptions
    )
    ml_predictor.train(df_train_WFx,
                       model_names=model_names,
                       perform_feature_scaling=True,
                       perform_feature_selection=True,
                       verbose=False,
                       cv=3,
                      )
    
    return ml_predictor

ml_predictor = train_wf(df_train_WFx)

Now using the model training_params that you passed in:
{}
After overwriting our defaults with your values, here are the final params that will be used to initialize the model:
{'presort': False, 'learning_rate': 0.1, 'warm_start': True}


E:\wei\logiciel\envs\alpheeenv\lib\site-packages\sklearn\base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


Running basic data cleaning
Performing feature scaling
Fitting DataFrameVectorizer
Performing feature selection


E:\wei\logiciel\envs\alpheeenv\lib\site-packages\sklearn\base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


Now using the model training_params that you passed in:
{}
After overwriting our defaults with your values, here are the final params that will be used to initialize the model:
{'presort': False, 'learning_rate': 0.1, 'warm_start': True}


********************************************************************************************
About to run GridSearchCV on the pipeline for several models to predict Production
Fitting 3 folds for each of 11 candidates, totalling 33 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:   33.6s
[Parallel(n_jobs=-1)]: Done  33 out of  33 | elapsed:  8.6min finished


The best CV score from our hyperparameter search (by default averaging across k-fold CV) for Production is:
-1.6723709955663562
The best params were
{'model': 'RandomForestRegressor'}
Here are all the hyperparameters that were tried:
Score in the following columns always refers to cross-validation score
+--------------+
|   mean_score |
|--------------|
|      -6.6824 |
|      -4.8241 |
|      -3.1706 |
|      -3.0666 |
|      -2.9155 |
|      -2.8224 |
|      -2.1925 |
|      -1.7879 |
|      -1.7618 |
|      -1.7005 |
|      -1.6724 |
+--------------+
Calculating feature responses, for advanced analytics.
Here are our feature responses for the trained model
+----+----------------+---------+-------------------+-------------------+-----------+-----------+
|    | Feature Name   |   Delta |   FR_Decrementing |   FR_Incrementing |   FRD_MAD |   FRI_MAD |
|----+----------------+---------+-------------------+-------------------+-----------+-----------|
| 30 | NWP1_12h_D-2_T |  0.1554 |     

E:\wei\logiciel\envs\alpheeenv\lib\site-packages\sklearn\base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


In [225]:
pred = pd.DataFrame({'pred': ml_predictor.predict(df_test_WFx), 'WF': wf_name, 'Time': df_test_WFx.index})

In [226]:
pred.head(2)

pred   WF                Time
0  5.913667  WF1 2019-01-16 01:00:00
1  5.913667  WF1 2019-01-16 02:00:00

## Run all WF

In [227]:
predictions = []
for wf_name in ['WF1', 'WF2', 'WF3', 'WF4', 'WF5', 'WF6']:
    df_train_WFx = create_data_WFx(wf_name, X_train, y_train)
    df_test_WFx = create_data_WFx(wf_name, X_test)
    ml_predictor = train_wf(df_train_WFx)
    pred = pd.DataFrame({'pred': ml_predictor.predict(df_test_WFx), 'WF': wf_name, 'Time': df_test_WFx.index})
    predictions.append(pred)
predictions = pd.concat(predictions)

Now using the model training_params that you passed in:
{}
After overwriting our defaults with your values, here are the final params that will be used to initialize the model:
{'presort': False, 'learning_rate': 0.1, 'warm_start': True}
Running basic data cleaning
Performing feature scaling
Fitting DataFrameVectorizer
Performing feature selection


E:\wei\logiciel\envs\alpheeenv\lib\site-packages\sklearn\base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


Now using the model training_params that you passed in:
{}
After overwriting our defaults with your values, here are the final params that will be used to initialize the model:
{'presort': False, 'learning_rate': 0.1, 'warm_start': True}


********************************************************************************************
About to run GridSearchCV on the pipeline for several models to predict Production
Fitting 3 folds for each of 11 candidates, totalling 33 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:   31.9s
[Parallel(n_jobs=-1)]: Done  33 out of  33 | elapsed:  8.7min finished


The best CV score from our hyperparameter search (by default averaging across k-fold CV) for Production is:
-1.6569212447732127
The best params were
{'model': 'ExtraTreesRegressor'}
Here are all the hyperparameters that were tried:
Score in the following columns always refers to cross-validation score
+--------------+
|   mean_score |
|--------------|
|      -6.6772 |
|      -4.9432 |
|      -3.9981 |
|      -3.0666 |
|      -2.8433 |
|      -2.8011 |
|      -2.2863 |
|      -1.7328 |
|      -1.7139 |
|      -1.6907 |
|      -1.6569 |
+--------------+
Calculating feature responses, for advanced analytics.
Here are our feature responses for the trained model
+----+----------------+---------+-------------------+-------------------+-----------+-----------+
|    | Feature Name   |   Delta |   FR_Decrementing |   FR_Incrementing |   FRD_MAD |   FRI_MAD |
|----+----------------+---------+-------------------+-------------------+-----------+-----------|
| 31 | NWP1_06h_D_T   |  0.1817 |       

...

E:\wei\logiciel\envs\alpheeenv\lib\site-packages\sklearn\base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


Now using the model training_params that you passed in:
{}
After overwriting our defaults with your values, here are the final params that will be used to initialize the model:
{'presort': False, 'learning_rate': 0.1, 'warm_start': True}


********************************************************************************************
About to run GridSearchCV on the pipeline for several models to predict Production
Fitting 3 folds for each of 11 candidates, totalling 33 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:   10.2s
[Parallel(n_jobs=-1)]: Done  33 out of  33 | elapsed:   37.7s finished


The best CV score from our hyperparameter search (by default averaging across k-fold CV) for Production is:
-0.5964380183811048
The best params were
{'model': 'ExtraTreesRegressor'}
Here are all the hyperparameters that were tried:
Score in the following columns always refers to cross-validation score
+--------------+
|   mean_score |
|--------------|
|      -3.5695 |
|      -3.4466 |
|      -2.8059 |
|      -1.2409 |
|      -1.2332 |
|      -1.2310 |
|      -0.7770 |
|      -0.6213 |
|      -0.6194 |
|      -0.6156 |
|      -0.5964 |
+--------------+
Calculating feature responses, for advanced analytics.
Here are our feature responses for the trained model
+----+----------------+---------+-------------------+-------------------+-----------+-----------+
|    | Feature Name   |   Delta |   FR_Decrementing |   FR_Incrementing |   FRD_MAD |   FRI_MAD |
|----+----------------+---------+-------------------+-------------------+-----------+-----------|
| 11 | NWP4_00h_D-1_V |  0.1486 |       

E:\wei\logiciel\envs\alpheeenv\lib\site-packages\sklearn\base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


In [228]:
predictions.head(2)

pred   WF                Time
0  4.5620  WF1 2019-01-16 01:00:00
1  4.5733  WF1 2019-01-16 02:00:00

In [229]:
df_test=X_test.merge(predictions, left_on=['WF', 'Time'], right_on=['WF', 'Time'])
pd.DataFrame({'ID': X_test.index, 'Production':df_test.pred}).to_csv('response.csv', index=False)

 score : 92.9056

# Grandiant Boosting Branch

In [238]:
from lightgbm import LGBMRegressor

def train_gbm_wf(df_train_WFx):
    
    lgbm_est = LGBMRegressor()
    X = df_train_WFx.loc[:, features]
    y = df_train_WFx.loc[:, target]

    lgbm_est.fit(X, y)

    return lgbm_est


In [240]:
pred = pd.DataFrame({'pred': lgbm_est.predict(df_test_WFx.loc[:, features]), 'WF': wf_name, 'Time': df_test_WFx.index})

In [242]:
predictions = []
for wf_name in ['WF1', 'WF2', 'WF3', 'WF4', 'WF5', 'WF6']:
    df_train_WFx = create_data_WFx(wf_name, X_train, y_train)
    df_test_WFx = create_data_WFx(wf_name, X_test)
    lgbm_est = train_gbm_wf(df_train_WFx)
    pred = pd.DataFrame({'pred': lgbm_est.predict(df_test_WFx.loc[:, features]), 
                         'WF': wf_name, 
                         'Time': df_test_WFx.index}) 
    predictions.append(pred)
predictions = pd.concat(predictions)
predictions.head(2)

In [244]:
df_test=X_test.merge(predictions, left_on=['WF', 'Time'], right_on=['WF', 'Time'])
pd.DataFrame({'ID': X_test.index, 'Production':df_test.pred}).to_csv('response_gbm.csv', index=False)

 score : 72

## optimisation

In [305]:
from sklearn.metrics import make_scorer

def loss_func(y_true, y_pred):
    #index = np.logical_not(np.isnan(y_true)) & (y_true>0)
    return 100*np.sum(np.abs(y_true - y_pred)) / np.sum(y_true)

score = make_scorer(loss_func, greater_is_better=False)

In [327]:
from sklearn.model_selection import RandomizedSearchCV
import copy

def train_gbm_wf2(df_train_WFx):

    distributions = dict(n_estimators=[50,100,150,200,300,400],
                         learning_rate=[0.01,0.03, 0.05, 0.1, 0.15, 0.2, 0.25])

    lgbm_est_opt = RandomizedSearchCV(LGBMRegressor(), 
                                  param_distributions = distributions, 
                                  random_state=20200229, 
                                  scoring=score,
                                  cv=4,
                                  verbose=3,
                                  n_jobs=-1)

    X = df_train_WFx.loc[:, features]
    y = df_train_WFx.loc[:, target]

    lgbm_est_opt.fit(X, y)
    return copy.deepcopy(lgbm_est_opt)

In [328]:
predictions = []
models = []
for wf_name in ['WF1', 'WF2', 'WF3', 'WF4', 'WF5', 'WF6']:
    df_train_WFx = create_data_WFx(wf_name, X_train, y_train)
    df_test_WFx = create_data_WFx(wf_name, X_test)
    lgbm_est2 = train_gbm_wf2(df_train_WFx)
    models.append(lgbm_est2)
    pred = pd.DataFrame({'pred': lgbm_est2.predict(df_test_WFx.loc[:, features]), 
                         'WF': wf_name, 
                         'Time': df_test_WFx.index}) 
    predictions.append(pred)
predictions = pd.concat(predictions)
predictions.head(2)

Fitting 4 folds for each of 10 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   36.9s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:  1.2min finished


Fitting 4 folds for each of 10 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   33.9s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:  1.1min finished


Fitting 4 folds for each of 10 candidates, totalling 40 fits


...

Fitting 4 folds for each of 10 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   35.7s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:  1.1min finished


Fitting 4 folds for each of 10 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   36.5s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:  1.1min finished


pred   WF                Time
0  1.927554  WF1 2019-01-16 01:00:00
1  1.912719  WF1 2019-01-16 02:00:00

In [332]:
df_test=X_test.merge(predictions, left_on=['WF', 'Time'], right_on=['WF', 'Time'])
pd.DataFrame({'ID': X_test.index, 'Production':df_test.pred}).to_csv('response_gbm_opt.csv', index=False)

 score : 70

In [333]:
cv_scores=[]
for model in models:
    cv_scores.append(model.cv_results_['mean_test_score'])
np.concatenate(cv_scores).mean()    

-39.12501300894234

In [334]:
for model in models:
    print(model.best_params_)

{'n_estimators': 150, 'learning_rate': 0.05}
{'n_estimators': 200, 'learning_rate': 0.05}
{'n_estimators': 300, 'learning_rate': 0.05}
{'n_estimators': 300, 'learning_rate': 0.03}
{'n_estimators': 300, 'learning_rate': 0.03}
{'n_estimators': 300, 'learning_rate': 0.03}


## feature engineering

In [413]:
from sklearn.pipeline import Pipeline, FeatureUnion

from sklearn.preprocessing import PowerTransformer
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
from sklearn.decomposition import PCA

from sklearn.model_selection import cross_val_score

pipe_p = Pipeline([
    ('gaussian', PowerTransformer()),
    ('pca', PCA(n_components=15)),
])

pipe_u = FeatureUnion([('acp', pipe_p),
                       ('sel_non_lin',SelectKBest(score_func=mutual_info_regression, k=10)),
                       ('sel_lin',SelectKBest(score_func=f_regression, k=10))])

transformed_data = pipe_u.fit_transform(df_train_WFx.loc[:, features].fillna(0), 
                                        df_train_WFx.loc[:, target])

month_ = df_train_WFx.index.month.values.reshape(-1,1)
hours_ = df_train_WFx.index.hour.values.reshape(-1,1)
prep_data = np.concatenate( (transformed_data,month_, hours_), axis=1)

In [414]:
cv_res = cross_val_score(estimator = LGBMRegressor(n_estimators=300, learning_rate=0.03),
                X = prep_data, 
                y = df_train_WFx.loc[:, target], 
                scoring=score, 
                cv =4, 
                n_jobs = -1)
np.mean(cv_res)

-33.54934622119134

In [445]:
from xgboost import XGBRegressor
from sklearn.linear_model import LassoCV, RidgeCV, LinearRegression
from sklearn.svm import LinearSVR, SVR

cv_res = cross_val_score(estimator = SVR(degree=2, C=0.9, epsilon=0.07),
                X = prep_data, 
                y = df_train_WFx.loc[:, target], 
                scoring=score, 
                cv =4, 
                n_jobs = -1)
np.mean(cv_res)

-32.740671774733

In [477]:
def train_svm_wf(df_train_WFx):

    transformed_data = pipe_u.fit_transform(df_train_WFx.loc[:, features].fillna(0), 
                                        df_train_WFx.loc[:, target])

    month_ = df_train_WFx.index.month.values.reshape(-1,1)
    hours_ = df_train_WFx.index.hour.values.reshape(-1,1)
    prep_data = np.concatenate( (transformed_data,month_, hours_), axis=1)
    
    
    
    distributions = dict(C=[0.5,0.6,0.7,0.8,0.9,1],
                         epsilon=[0.05,0.06, 0.07, 0.08, 0.09, 0.1, 0.11])

    svm_est_opt = RandomizedSearchCV(SVR(), 
                                  param_distributions = distributions, 
                                  scoring=score,
                                  cv=4,
                                  verbose=3,
                                  n_jobs=-1)
    
    svm_est_opt = SVR(degree=2, C=0.9, epsilon=0.07)

    X = prep_data
    y = df_train_WFx.loc[:, target]

    svm_est_opt.fit(X, y)
    return copy.deepcopy(svm_est_opt)

In [475]:
predictions = []
models = []
for wf_name in ['WF1', 'WF2', 'WF3', 'WF4', 'WF5', 'WF6']:
    df_train_WFx = create_data_WFx(wf_name, X_train, y_train)
    df_test_WFx = create_data_WFx(wf_name, X_test)
    svm_est = train_svm_wf(df_train_WFx)
    models.append(svm_est)
    pred = pd.DataFrame({'pred': svm_est.predict(df_test_WFx.loc[:, features].fillna(0)), 
                         'WF': wf_name, 
                         'Time': df_test_WFx.index}) 
    predictions.append(pred)
predictions = pd.concat(predictions)
predictions.head(2)

pred   WF                Time
0  1.315223  WF1 2019-01-16 01:00:00
1  1.248588  WF1 2019-01-16 02:00:00

In [472]:
cv_scores=[]
for model in models:
    cv_scores.append(model.cv_results_['mean_test_score'])
np.concatenate(cv_scores).mean()    

-79.67965976783672

In [473]:
cv_scores

[array([-86.81232162, -83.72622612, -83.73081405, -86.80425261,
        -83.26424456, -83.26719772, -83.26481389, -84.96377987,
        -86.79716949, -84.28738886]),
 array([-77.89626966, -77.9830144 , -77.99565752, -77.9915717 ,
        -78.01571259, -77.98844862, -77.93084708, -77.98078164,
        -77.9567673 , -77.97131306]),
 array([-76.3397417 , -76.31998589, -76.48264374, -76.33898117,
        -76.36354736, -76.25851159, -76.2950147 , -76.51068602,
        -76.4660717 , -76.24929173]),
 array([-79.13744698, -79.08762208, -79.07386028, -79.05707455,
        -79.12832007, -79.08724096, -79.11551802, -79.09407489,
        -79.18020388, -79.09941831]),
 array([-80.83166061, -80.79606182, -80.93090616, -80.80350608,
        -80.91694305, -81.02105489, -80.7391661 , -80.89847453,
        -80.80212356, -81.03441513]),
 array([-79.30444992, -79.28596161, -79.11459455, -78.96438792,
        -78.67800519, -79.309271  , -78.82100031, -79.11819788,
        -79.12776893, -78.96778887])]

## Stacking

In [449]:
from sklearn.ensemble import StackingRegressor, RandomForestRegressor

In [464]:
estimators = [
    ('lgbm', LGBMRegressor(n_estimators=300, learning_rate=0.03)),
    ('svr', SVR(degree=2, C=0.9, epsilon=0.07))
    ]
est_stacking = StackingRegressor(
    estimators=estimators,
    final_estimator=LassoCV(),
    cv=4,
)

In [465]:
cv_res = cross_val_score(estimator =est_stacking,
                X = prep_data, 
                y = df_train_WFx.loc[:, target], 
                scoring = score, 
                cv = 4, 
                n_jobs = -1)
np.mean(cv_res)

-32.53837343289987